# Laboratorio 4

In [43]:
# librerias necesarias
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Concatenate
from tensorflow.keras.layers import Input
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, concatenate


### Importación de datos

In [5]:
num_words = 50000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)


### Preprocesamiento

In [7]:
# 1 secuenciar y rellenar criticas para que sean uniformes
max_sequence_length = 200  # Longitud máxima de la secuencia de palabras

x_train = pad_sequences(x_train, maxlen=max_sequence_length)
x_test = pad_sequences(x_test, maxlen=max_sequence_length)

# 2 crear una funcion para extraer caracteristicas de una critica
# recibe una critica como parametro
def get_review_features(review_text):
    
    # Diccionario que mapea índices de palabras a palabras
    word_index_reverse = {index: word for word, index in imdb.get_word_index().items()}

    # Convierte la secuencia numérica en texto claro
    review_text = " ".join([word_index_reverse.get(index, "") for index in review_text])
    # Calcular la longitud de la crítica
    review_length = len(review_text.split())

    # crear nuestro indice de palabras
    positive_words = ["good", "excellent", "outstanding", "fantastic", "awesome", "funny", "hilarious", "cool"]
    negative_words = ["bad", "poor", "terrible", "awful", "horrible", "boring", "sleepy", "fool", "silly", "goofy"]

    positive_count = 0
    negative_count = 0

    # Contar palabras positivas y negativas
    for word in review_text.split():
        if word in positive_words:
            positive_count += 1
        if word in negative_words:
            negative_count += 1
    # Calcular la proporción de palabras positivas/negativas
    if (positive_count + negative_count) > 0:
        positivity_ratio = positive_count / (positive_count + negative_count)
    else:
        positivity_ratio = 0.0

    return review_length, positivity_ratio


### Modelos

In [8]:
# normalizar features extra
rl = []
pr = []

for review in x_train:
    uwu, owo = get_review_features(review)

    rl.append(uwu)
    pr.append(owo)

# Convierte las listas de características en arreglos numpy
feature_1_train = np.array(rl)
feature_2_train = np.array(pr)


In [9]:
print(feature_1_train)

[200 189 141 ... 184 150 153]


In [10]:
rl = []
pr = []

for review in x_test:
    uwu, owo = get_review_features(review)

    rl.append(uwu)
    pr.append(owo)

# Convierte las listas de características en arreglos numpy
feature_1_test = np.array(rl)
feature_2_test = np.array(pr)


In [33]:
# Preparar las características adicionales
new_features_train = np.stack([feature_1_train, feature_2_train], axis=1)
new_features_test = np.stack([feature_1_test, feature_2_test], axis=1)

(25000, 2)


LSTM

In [44]:
# Definición de la estructura del modelo

# Entrada para la secuencia de palabras
word_input = Input(shape=(200,), dtype='int32', name='word_input')

# Capa de incrustación (embedding) para convertir palabras en vectores
word_embedding = Embedding(input_dim=50000, output_dim=128, input_length=200)(word_input)

# Capa LSTM para procesar la secuencia de palabras
lstm_out = LSTM(32)(word_embedding)

# Entrada para las características adicionales
additional_input = Input(shape=(2,), name='additional_input')

# Combinar las salidas de la capa LSTM y las características adicionales
merged = concatenate([lstm_out, additional_input])

# Capa densa de salida con activación sigmoide para clasificación binaria
output = Dense(1, activation='sigmoid', name='output')(merged)

# Crear el modelo
model_simple = Model(inputs=[word_input, additional_input], outputs=[output])

# Compilar el modelo con optimizador 'adam' y función de pérdida 'binary_crossentropy'
model_simple.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con los datos de entrenamiento
model_simple.fit([x_train, new_features_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 70s 109ms/step - loss: 0.7727 - accuracy: 0.6520 - val_loss: 0.4208 - val_accuracy: 0.8230
Epoch 2/10
625/625 [==============================] - 68s 109ms/step - loss: 0.3202 - accuracy: 0.8675 - val_loss: 0.4067 - val_accuracy: 0.8258
Epoch 3/10
625/625 [==============================] - 69s 110ms/step - loss: 0.1737 - accuracy: 0.9369 - val_loss: 0.3700 - val_accuracy: 0.8556
Epoch 4/10
625/625 [==============================] - 68s 108ms/step - loss: 0.0899 - accuracy: 0.9712 - val_loss: 0.4891 - val_accuracy: 0.8514
Epoch 5/10
625/625 [==============================] - 69s 111ms/step - loss: 0.0506 - accuracy: 0.9844 - val_loss: 0.5082 - val_accuracy: 0.8548
Epoch 6/10
625/625 [==============================] - 71s 114ms/step - loss: 0.0321 - accuracy: 0.9912 - val_loss: 0.5856 - val_accuracy: 0.8490
Epoch 7/10
625/625 [==============================] - 72s 116ms/step - loss: 0.0278 - accuracy: 0.9916 - val_loss: 0.6457 - val_ac

In [46]:
score = model_simple.evaluate([x_test, new_features_test], y_test, batch_size=32)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

782/782 [==============================] - 10s 12ms/step - loss: 0.9097 - accuracy: 0.8319
Test loss: 0.9097073674201965
Test accuracy: 0.8318799734115601


Arquitectura mas compleja

In [51]:
# Entrada para la secuencia de palabras
word_input = Input(shape=(200,), dtype='int32', name='word_input')

# Capa de incrustación para representar palabras como vectores densos
word_embedding = Embedding(input_dim=50000, output_dim=128, input_length=200)(word_input)

# Capa LSTM para procesar la secuencia de palabras con regularización Dropout
lstm_out1 = LSTM(64, return_sequences=True)(word_embedding)  # LSTM con retorno de secuencias
dropout1 = Dropout(0.4)(lstm_out1)  # Dropout para regularización

# Capa LSTM adicional
lstm_out2 = LSTM(32)(dropout1)  # LSTM sin retorno de secuencias
dropout2 = Dropout(0.4)(lstm_out2)  # Dropout para regularización

# Entrada para las características adicionales (aquí asumimos 3 características)
additional_input = Input(shape=(2,), name='additional_input')

# Combinar las salidas de las capas LSTM y las características adicionales
merged = concatenate([dropout2, additional_input])

# Capas densas adicionales para procesar la información combinada
dense1 = Dense(32, activation='relu')(merged)
dense2 = Dense(16, activation='relu')(dense1)

# Capa densa de salida con activación sigmoide para clasificación binaria
output = Dense(1, activation='sigmoid', name='output')(dense2)

# Crear y compilar el modelo
model_complex = Model(inputs=[word_input, additional_input], outputs=[output])
model_complex.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [52]:
# Entrenar y evaluar el modelo complejo
history_complex = model_complex.fit([x_train, new_features_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

score_complex = model_complex.evaluate([x_test, new_features_test], y_test, batch_size=32)
print('Test loss:', score_complex[0])
print('Test accuracy:', score_complex[1])

Epoch 1/10
625/625 [==============================] - 91s 141ms/step - loss: 0.4778 - accuracy: 0.7775 - val_loss: 0.3359 - val_accuracy: 0.8598
Epoch 2/10
625/625 [==============================] - 87s 139ms/step - loss: 0.2050 - accuracy: 0.9238 - val_loss: 0.3072 - val_accuracy: 0.8716
Epoch 3/10
625/625 [==============================] - 87s 139ms/step - loss: 0.1075 - accuracy: 0.9638 - val_loss: 0.5119 - val_accuracy: 0.8532
Epoch 4/10
625/625 [==============================] - 87s 140ms/step - loss: 0.0564 - accuracy: 0.9816 - val_loss: 0.5767 - val_accuracy: 0.8628
Epoch 5/10
625/625 [==============================] - 87s 139ms/step - loss: 0.0591 - accuracy: 0.9803 - val_loss: 0.4722 - val_accuracy: 0.8456
Epoch 6/10
625/625 [==============================] - 93s 148ms/step - loss: 0.0388 - accuracy: 0.9886 - val_loss: 0.6287 - val_accuracy: 0.8558
Epoch 7/10
625/625 [==============================] - 94s 151ms/step - loss: 0.0317 - accuracy: 0.9900 - val_loss: 0.7722 - val_ac

782/782 [==============================] - 22s 28ms/step - loss: 0.9552 - accuracy: 0.8404
Test loss: 0.9551502466201782
Test accuracy: 0.840399980545044
